# Gráficos de Círculo, Caja, Dispersión




# Explorando Conjunto de Datos con *pandas* y *Matplotlib*<a id="0"></a>



Conjunto de datos: Inmigración a Canada desde 1980 a 2013 - [International migration flows to and from selected countries - The 2015 revision](http://www.un.org/en/development/desa/population/migration/data/empirical2/migrationflows.asp) desde el sitio ewb de las Naciones Unidas.

El conjunto de datos contiene datos anuales sobre los flujos de migrantes internacionales registrados por los países de destino. Los datos presentan tanto las entradas como las salidas según el lugar de nacimiento, ciudadanía o lugar de residencia anterior/próxima tanto para extranjeros como para nacionales. Este laboratorio se concentra en los datos de inmigración canadiense.

# Descarga y Preparación de Datos <a id="2"></a>


Lo primero que hay que realizar es la instalación de **openpyxl** (antiguamente conocido como **xlrd**) que es un módulo que **pandas** requiere para leer archivos Excel.

In [ ]:
!pip install openpyxl

A continuación, se procederá a cargar los dos módulos claves para hacer análisis de datos **pandas** y
 **numpy**.

In [ ]:
import numpy as np  # util para computación científica en Python
import pandas as pd # biblioteca que contienen la estructura de datos de uso principal

Ahora, se carga el conjunto de datos con los datos de la inmigración en Canadá usando el método de **pandas** `read_excel()`.

In [ ]:
df_can = pd.read_excel(
    'datos/Canada.xlsx',
    sheet_name='Canada by Citizenship',
    skiprows=range(20),
    skipfooter=2)

print('Datos leídos y cargados en un dataframe de pandas!')

Mostrar las primeras 5 filas del conjunto de datos usando la función `head()`.


In [ ]:
df_can.head()

Descubrir cuantas entradas hay en el conjunto de datos.


In [ ]:
# dimensión del dataframe (filas, columnas)
df_can.shape  

Limpiar datos. Se realizarán algunas modificaciones al conjunto de datos original para que sea más fácil crear visualizaciones. La explicación de estas modificaciones están detalladas en los laboratorios anteriores *"Introducción a Matplotlib y Gráficos de línea"* y *"Gráficos de Área, Barras e Histogramas"*.


In [ ]:
# limpiar el conjunto de datos para eliminar columnas innecesarias (por ejemplo, REG) 
df_can.drop(['AREA', 'REG', 'DEV', 'Type', 'Coverage'], axis=1, inplace=True)

# cambiar el nombre de algunas de las columnas para que tengan sentido
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)

# para mantener la coherencia, asegúrese de que todas las etiquetas de las columnas sean de tipo cadena de caracteres o string
df_can.columns = list(map(str, df_can.columns))

# establecer el nombre del país como índice. Esto es útil para buscar rápidamente países utilizando el método .loc
df_can.set_index('Country', inplace=True)

# agregar la columna total
df_can['Total'] = df_can.sum(axis=1, numeric_only=True)

# finalmente, se crea una lista de años desde 1980 - 2013, esto será útil cuando comencemos a graficar los datos
years = list(map(str, range(1980, 2014)))
print('Dimensión de los datos:', df_can.shape)

# Visualización de datos usando Matplotlib<a id="4"></a>


Importar la biblioteca `matplotlib`.


In [ ]:
# se hará uso del despliegue de figuras en línea o dentro de Jupyter Notebook
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt


mpl.style.use('ggplot')  # opcional: para estilo similar a ggplot

# verificar la versión de Matplotlib
print('Matplotlib version: ', mpl.__version__)

# Gráfico de Círculo <a id="6"></a>

Un *gráfico de círculo* es un gráfico que muestra proporciones numéricas al dividir un círculo (o un pastel) en porciones proporcionales. Estos gráficos se utilizan ampliamente en los negocios y los medios. Podemos crear gráficos de círculos en Matplotlib pasando la palabra clave `kind=pie`.

Se usará un gráfico de círculo para explorar la proporción (porcentaje) de nuevos inmigrantes agrupados por continentes durante todo el período de 1980 a 2013.


Paso 1: Reunir los datos.

Se usará el método `groupby` de *pandas* para resumir los datos de inmigración por `Continent`. El proceso general de `groupby` implica los siguientes pasos:

1. **Dividir:** Dividir los datos en grupos según algunos criterios.
2. **Aplicar:** Aplicar una función a cada grupo de forma independiente:
     .sum()
     .count()
     .mean()
     .std()
     .aggregate()
     .apply()
     .etc..
3. **Combinar:** combinar los resultados en una estructura de datos.


![groupby](imagenes/groupby.png)


In [ ]:
# agrupar países por continente y aplicar la función sum() 
df_continentes = df_can.groupby('Continent', axis=0).sum()

# la salida del método groupby es un objeto 'groupby'. 
# y esta no se puede usar hasta que se le aplique una función (por ejemplo, .sum())
print(type(df_can.groupby('Continent', axis=0)))

df_continentes.head()

Paso 2: Graficar los datos. Se debe pasar la palabra clave `kind = 'pie'`, junto con los siguientes parámetros adicionales:

*   `autopct` -  es una cadena o función utilizada para etiquetar las porciones con su valor numérico. La etiqueta se colocará dentro de la porción. Si es una cadena de formato, la etiqueta será `fmt%pct`.
*   `startangle` - gira el inicio del gráfico en grados de ángulo en sentido contrario a las agujas del reloj desde el eje x.
*   `shadow` - Dibuja una sombra debajo del pastel (para dar una sensación de 3D).


In [ ]:
# autopct crear %, el ángulo de inicio representa el punto de inicio
df_continentes['Total'].plot(kind='pie',
                            figsize=(5, 6),
                            autopct='%1.1f%%', # añadir en porcentajes
                            startangle=90,     # ángulo de inicio 90° (África)
                            shadow=True,       # añadir sombra     
                            )

plt.title('Inmigración a Canadá por Continente [1980 - 2013]')
plt.axis('equal') # Establece el gráfico para que parezca un círculo.

plt.show()

La imagen anterior no es muy clara, los números y el texto se superponen en algunos casos. Se deben hacer algunas modificaciones para mejorar la imagen:

* Eliminar las etiquetas de texto en el gráfico pasando `legend` y agregar una leyenda separada usando `plt.legend()`.
* Mover los porcentajes para que queden fuera del gráfico pasando el parámetro `pctdistance`.
* Definir un conjunto personalizado de colores para los continentes pasando el parámetro `colors`.
* **Explotar** el gráfico para enfatizar los tres continentes más bajos (África, América del Norte y América Latina y el Caribe) pasando el parámetro `explode`.


In [ ]:
lista_colores = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightgreen', 'pink']
lista_explotar = [0.1, 0, 0, 0, 0.1, 0.1] # relación para cada continente del inclinado de cada porción.
df_continentes['Total'].plot(kind='pie',
                            figsize=(15, 6),
                            autopct='%1.1f%%', 
                            startangle=90,    
                            shadow=True,       
                            labels=None,           # desactiva las etiquetas en el gráfico
                            pctdistance=1.12,      # relación entre el centro de cada sector circular y el inicio del texto generado por autopct 
                            colors=lista_colores,  # añadir colores personalizados
                            explode=lista_explotar # explotar los 3 continentes más bajos
                            )

# aumentar el título en un 12% para que coincida con pctdistance
plt.title('Inmigración a Canada por Continente [1980 - 2013]', y=1.12) 

plt.axis('equal') 

# añadir leyenda
plt.legend(labels=df_continentes.index, loc='upper left') 

plt.show()

# Gráfico de caja <a id="8"></a>

Un `gráfico de caja` es una forma de representar estadísticamente la *distribución* de los datos a través de cinco dimensiones principales:

* **Mínimo:** El número más pequeño en el conjunto de datos excluyendo los valores atípicos.
* **Primer cuartil:** Número medio entre el `mínimo` y la `mediana`.
* **Segundo cuartil (mediana):** Número medio del conjunto de datos (ordenado).
* **Tercer cuartil:** Número medio entre `mediana` y `máximo`.
* **Máximo:** El número más grande en el conjunto de datos excluyendo los valores atípicos.

![caja](imagenes/caja.png)


Para hacer un gráfico de caja o `boxplot`, se debe usar `kind=box` en el método `plot` invocado en una serie *pandas* o dataframe.

Generar un gráfico de caja para los inmigrantes japoneses entre 1980 y 2013.


Paso 1: Obtener el subconjunto del conjunto de datos. Si bien se están extrayendo los datos de un solo país, estos se obtendrán como un dataframe. Así, se llamará al método `dataframe.describe()` para ver los percentiles.

In [ ]:
# para obtener el dataframe, colocar corchetes adicionales alrededor de 'Japón'.
df_japon = df_can.loc[['Japan'], years].transpose()
df_japon.head()

Paso 2: Graficar configurando `kind='box'`.


In [ ]:
df_japon.plot(kind='box', figsize=(8, 6))

plt.title('Gráfico de Caja de Inmigrantes Japoneses entre 1980 - 2013')
plt.ylabel('Número de Inmigrantes')

plt.show()

Algunas observaciones claves de la gráfica anterior:

1. El número mínimo de inmigrantes es de alrededor de 200 (min), el número máximo es de alrededor de 1300 (max) y el número medio de inmigrantes es de alrededor de 900 (mediana).
2. El 25% de los años del período 1980 - 2013 tuvo un recuento anual de inmigrantes de \~500 o menos (primer cuartil).
3. El 75% de los años del período 1980 - 2013 tuvo un recuento anual de inmigrantes de \~1100 o menos (tercer cuartil).

Podemos ver los números reales llamando al método `describe()` en el dataframe.


In [ ]:
df_japon.describe()

# Gráficos de Dispersión <a id="10"></a>

Un *gráfico de dispersión* es útil para comparar variables entre sí. Los gráficos de *dispersión* se parecen a los *gráficos de líneas* en el sentido de que mapean variables independientes y dependientes en un gráfico de 2 dimensiones. Si bien los puntos de datos están conectados entre sí por una línea en un gráfico de líneas, no están conectados en un gráfico de dispersión. Se considera que los datos en un diagrama de dispersión expresan una tendencia. Con un análisis más profundo usando herramientas como la regresión, podemos calcular matemáticamente esta relación y usarla para predecir tendencias fuera del conjunto de datos.

Para entender su funcionamiento se va a trabajar en lo siguiente:

Usando un *gráfico de dispersión*, visualizar la tendencia de la inmigración total a Canadá (todos los países combinados) para los años 1980 - 2013.


Paso 1: Obtener el conjunto de datos. Como se requiere usar la relación entre `años` y `total de población`, se convertirán los `años` al tipo `int`.


In [ ]:
# usar el método sum() para obtener la población total por año
df_total = pd.DataFrame(df_can[years].sum(axis=0))

# cambiar los años al tipo entero (útil para la regresión más adelante)
df_total.index = map(int, df_total.index)

# restablecer el índice para volver a colocarlo como una columna en el dataframe df_total
df_total.reset_index(inplace = True)

# renombrar columnas
df_total.columns = ['year', 'total']

# mostrar el dataframe resultante
df_total.head()

Paso 2: Graficar los datos. En `Matplotlib`, se puede crear un conjunto de gráficos `scatter` configurando `kind='scatter'` como argumento del tipo de gráfico. También se  necesita pasar las palabras clave `x` e `y` para especificar las columnas que van en el eje x e y.

In [ ]:
df_total.plot(kind='scatter', x='year', y='total', figsize=(10, 6), color='darkblue')

plt.title('Inmigración Total a Canadá entre 1980 - 2013')
plt.xlabel('Años')
plt.ylabel('Número de Inmigrantes')

plt.show()

Notar cómo el diagrama de dispersión no conecta los puntos de datos entre sí. Se puede observar una tendencia al alza en los datos: a medida que pasan los años, aumenta el número total de inmigrantes. También, se puede analizar matemáticamente esta tendencia al alza utilizando una línea de regresión (línea de mejor ajuste).


Para trazar una línea de mejor ajuste y usarla para predecir el número de inmigrantes en 2015 se deben seguir los siguientes pasos.

Paso 1: Obtener la ecuación de la línea de mejor ajuste. Para esto se usará el método `polyfit()` de **Numpy** pasando los siguientes parámetros:

* `x`: coordenadas x de los datos.
* `y`: coordenadas y de los datos.
* `deg`: Grado de ajuste del polinomio. 1 = lineal, 2 = cuadrático, y así sucesivamente.


In [ ]:
x = df_total['year']      # año en el eje x
y = df_total['total']     # total en el eje y
ajuste = np.polyfit(x, y, deg=1)

ajuste

La salida es un arreglo con los coeficientes polinómicos, las potencias más altas primero. Dado que estamos trazando una regresión lineal `y= a * x + b`, nuestra salida tiene 2 elementos `[5.56709228e+03, -1.09261952e+07]` con la pendiente en la posición 0 y la intersección en la posición 1.

Paso 2: Trazar la línea de regresión en el *diagrama de dispersión*.


In [ ]:
df_total.plot(kind='scatter', x='year', y='total', figsize=(10, 6), color='darkblue')

plt.title('Inmigración Total a Canadá entre 1980 - 2013')
plt.xlabel('Años')
plt.ylabel('Número de Inmigrantes')

# trazar la línea del mejor ajuste
plt.plot(x, ajuste[0] * x + ajuste[1], color='red') # recordar que x son los años
plt.annotate('y={0:.0f} x + {1:.0f}'.format(ajuste[0], ajuste[1]), xy=(2000, 150000))

plt.show()

# imprimir la línea de mejor ajuste
'Número de Inmigrantes = {0:.0f} * Año + {1:.0f}'.format(ajuste[0], ajuste[1]) 

Al usar la ecuación de recta de mejor ajuste, se puede estimar el número de inmigrantes en 2015:

```python
Nº Inmigrantes = 5567 * Año - 10926195
Nº Inmigrantes = 5567 * 2015 - 10926195
Nº Inmigrantes = 291.310
```

En comparación con el informe anual de 2016 de Citizenship and Immigration Canada (CIC) (https://www.canada.ca/en/immigration-refugees-citizenship/corporate/publications-manuals/annual-report-parliament-immigration-2016.html), se puede ver observar que Canadá aceptó 271.845 inmigrantes en 2015, que es bastante bueno. El valor estimado de 291,310 está dentro del 7% del número real, lo cual es bastante bueno considerando que nuestros datos originales provienen de las Naciones Unidas (y pueden diferir ligeramente de los datos de CIC).
